# Azure SQL Graph Model Support

This sample creates a graph schema, as showed in Figure 1, for a hypothetical social network that has People, Restaurant and City nodes. These nodes are connected to each other using Friends, Likes, LivesIn and LocatedIn edges.

![Sample schema with restaurant, city, person nodes and LivesIn, LocatedIn, Likes edges.](https://docs.microsoft.com/en-us/sql/relational-databases/graphs/media/person-cities-restaurants-tables.png?view=sql-server-ver15)

## Create NODE tables

In [4]:
CREATE TABLE Person (
  ID INTEGER PRIMARY KEY,
  [Name] VARCHAR(100)
) AS NODE;

CREATE TABLE Restaurant (
  ID INTEGER NOT NULL,
  [Name] VARCHAR(100),
  [City] VARCHAR(100)
) AS NODE;

CREATE TABLE City (
  ID INTEGER PRIMARY KEY,
  [Name] VARCHAR(100),
  [StateName] VARCHAR(100)
) AS NODE;

Commands completed successfully.

Total execution time: 00:00:00.067

## Create EDGE tables

In [5]:
CREATE TABLE Likes (rating INTEGER) AS EDGE;
CREATE TABLE FriendOf AS EDGE;
CREATE TABLE LivesIn AS EDGE;
CREATE TABLE LocatedIn AS EDGE;

Commands completed successfully.

Total execution time: 00:00:00.052

## Insert data into NODE tables
Inserting into a node table is same as inserting into a regular table


In [6]:
INSERT INTO Person (Id, name)
	VALUES (1, 'John')
		 , (2, 'Mary')
		 , (3, 'Alice')
		 , (4, 'Jacob')
		 , (5, 'Julie');

INSERT INTO Restaurant (Id, name, city)
	VALUES (1, 'Taco Dell','Bellevue')
		 , (2, 'Ginger and Spice','Seattle')
		 , (3, 'Noodle Land', 'Redmond');

INSERT INTO City (Id, name, stateName)
	VALUES (1,'Bellevue','wa')
		 , (2,'Seattle','wa')
		 , (3,'Redmond','wa');

(5 rows affected)

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.072

## Insert into edge table

While inserting into an edge table, you need to provide the __$node_id__ from __$from_id__ and __$to_id__ system columns.


In [7]:
/* Insert which restaurants each person likes */
INSERT INTO likes 
	VALUES ((SELECT $node_id FROM Person WHERE ID = 1), (SELECT $node_id FROM Restaurant WHERE ID = 1), 9)
		 , ((SELECT $node_id FROM Person WHERE ID = 2), (SELECT $node_id FROM Restaurant WHERE ID = 2), 9)
		 , ((SELECT $node_id FROM Person WHERE ID = 3), (SELECT $node_id FROM Restaurant WHERE ID = 3), 9)
		 , ((SELECT $node_id FROM Person WHERE ID = 4), (SELECT $node_id FROM Restaurant WHERE ID = 3), 9)
		 , ((SELECT $node_id FROM Person WHERE ID = 5), (SELECT $node_id FROM Restaurant WHERE ID = 3), 9);

/* Associate in which city live each person*/
INSERT INTO livesIn 
	VALUES ((SELECT $node_id FROM Person WHERE ID = 1), (SELECT $node_id FROM City WHERE ID = 1))
		 , ((SELECT $node_id FROM Person WHERE ID = 2), (SELECT $node_id FROM City WHERE ID = 2))
		 , ((SELECT $node_id FROM Person WHERE ID = 3), (SELECT $node_id FROM City WHERE ID = 3))
		 , ((SELECT $node_id FROM Person WHERE ID = 4), (SELECT $node_id FROM City WHERE ID = 3))
		 , ((SELECT $node_id FROM Person WHERE ID = 5), (SELECT $node_id FROM City WHERE ID = 1));

/* Insert data where the restaurants are located */
INSERT INTO locatedIn 
	VALUES ((SELECT $node_id FROM Restaurant WHERE ID = 1), (SELECT $node_id FROM City WHERE ID =1))
		 , ((SELECT $node_id FROM Restaurant WHERE ID = 2), (SELECT $node_id FROM City WHERE ID =2))
		 , ((SELECT $node_id FROM Restaurant WHERE ID = 3), (SELECT $node_id FROM City WHERE ID =3));

/* Insert data into the friendOf edge */
INSERT INTO friendOf 
	VALUES ((SELECT $NODE_ID FROM Person WHERE ID = 1), (SELECT $NODE_ID FROM Person WHERE ID = 2))
		 , ((SELECT $NODE_ID FROM Person WHERE ID = 2), (SELECT $NODE_ID FROM Person WHERE ID = 3))
		 , ((SELECT $NODE_ID FROM Person WHERE ID = 3), (SELECT $NODE_ID FROM Person WHERE ID = 1))
		 , ((SELECT $NODE_ID FROM Person WHERE ID = 4), (SELECT $NODE_ID FROM Person WHERE ID = 2))
		 , ((SELECT $NODE_ID FROM Person WHERE ID = 5), (SELECT $NODE_ID FROM Person WHERE ID = 4));

(5 rows affected)

(5 rows affected)

(3 rows affected)

(5 rows affected)

Total execution time: 00:00:00.079

## Query Graph

Find Restaurants that John likes

In [8]:
SELECT Restaurant.name
FROM Person, likes, Restaurant
WHERE MATCH (Person-(likes)->Restaurant)
AND Person.name = 'John';

(1 row affected)

Total execution time: 00:00:00.168

name
Taco Dell


Find Restaurants that John's friends like

In [9]:
SELECT Restaurant.name 
FROM Person person1, Person person2, likes, friendOf, Restaurant
WHERE MATCH(person1-(friendOf)->person2-(likes)->Restaurant)
AND person1.name='John';


(1 row affected)

Total execution time: 00:00:00.955

name
Ginger and Spice


Find people who like a restaurant in the same city they live in

In [10]:
SELECT Person.name
FROM Person, likes, Restaurant, livesIn, City, locatedIn
WHERE MATCH (Person-(likes)->Restaurant-(locatedIn)->City AND Person-(livesIn)->City);

(4 rows affected)

Total execution time: 00:00:00.075

name
John
Mary
Alice
Jacob


## Cleanup

In [11]:
DROP TABLE Person, Restaurant, City;
DROP TABLE Likes, FriendOf, LivesIn, LocatedIn;

Commands completed successfully.

Total execution time: 00:00:00.105